# CWT hyperparameters tuning

Use GPU T4 x 2  
When using GP100, there are XLA errors.  

5 channels (LT, RT, LP, RP, C).

Implementing tf.keras.metrics.KLDivergence().



In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers, regularizers
import keras_tuner as kt
import sys
import os


# ----------------------------------------
# Flags for working on my different machines.
# flag_kaggle = True
flag_FW = True
# flag_LN = True

try:
    if flag_kaggle:
        sys.path.insert(0, '/kaggle/input/hms-lib')
        base_dir = '/kaggle/input/hms-harmful-brain-activity-classification'
        devset_dir = '/kaggle/input/hms-cwt-scalograms-single-numpy-v1'
        output_dir = ''
except:
    pass 

try:
    if flag_FW:
        sys.path.insert(0, '../lib')
        base_dir = '../../kaggle_data/hms'
        devset_dir = '../data'
        output_dir = 'results/'
except:
    pass 

try:
    if flag_LN:
        sys.path.insert(0, '../lib')
        base_dir = '../../data/hms'
        devset_dir = '../data'
        output_dir = 'results/'
except:
    pass 
# ----------------------------------------

from KLmetric import score

# path_train = f'{devset_dir}/05_single_cwt_v1_train.npy'
# path_train_items = f'{devset_dir}/05_single_cwt_v1_train_items.npy'
# path_val = f'{devset_dir}/05_single_cwt_v1_val.npy'
# path_val_items = f'{devset_dir}/05_single_cwt_v1_val_items.npy'
# path_test = f'{devset_dir}/05_single_cwt_v1_test.npy'
# path_test_items = f'{devset_dir}/05_single_cwt_v1_test_items.npy'

path_train = f'{devset_dir}/05_reduced_single_cwt_v1_train.npy'
path_train_items = f'{devset_dir}/05_reduced_single_cwt_v1_train_items.npy'
path_val = f'{devset_dir}/05_reduced_single_cwt_v1_val.npy'
path_val_items = f'{devset_dir}/05_reduced_single_cwt_v1_val_items.npy'
path_test = f'{devset_dir}/05_reduced_single_cwt_v1_test.npy'
path_test_items = f'{devset_dir}/05_reduced_single_cwt_v1_test_items.npy'


## Data generators

In [20]:
#
# Data generator for training.
#
# coefficients of cwt's arrays
# 5 channels (LP, RP, LT, RP, C)
#

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        # self.n_freqs = 40

        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size, self.n_classes), dtype=float)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]
            # Store solution
            y[i,:] = item[-6:]

        return X, y


#
# Test Data generator for predicting
# 

class TestDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=False):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        # pass 
        
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]

        return X


In [6]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

training_generator = DataGenerator(path_train_items, path_train , **params)
validation_generator = DataGenerator(path_val_items, path_val, **params)

print("Observations in training set:", training_generator.__len__()*params['batch_size'])
print("Observations in validation set:", validation_generator.__len__()*params['batch_size'])


Observations in training set: 512
Observations in validation set: 128


## HP tuning

In [7]:
input_shape = training_generator.get_dim()
num_classes = 6

def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.Input(shape=input_shape))

    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(layers.Conv2D(32, 3, activation='relu', padding="same"))
    
    model.add(layers.MaxPooling2D((3, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(num_classes, activation='softmax'))



    # model.add(layers.Flatten())
    # # Tune the number of layers.
    # for i in range(hp.Int("num_layers", 1, 3)):
    #     model.add(
    #         layers.Dense(
    #             # Tune number of units separately.
    #             units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
    #             activation=hp.Choice("activation", ["relu", "tanh"]),
    #         )
    #     )
    # if hp.Boolean("dropout"):
    #     model.add(layers.Dropout(rate=0.25))
    # model.add(layers.Dense(10, activation="softmax"))
    # # learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    learning_rate = 0.01

    opt = keras.optimizers.SGD(
        learning_rate=learning_rate,
        momentum=0.01,
        )
    
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.KLDivergence(),
        metrics=[tf.keras.metrics.KLDivergence()])

    return model


In [8]:
tuner = kt.Hyperband(model_builder,
                    #  objective='val_accuracy',
                     objective=kt.Objective("val_kullback_leibler_divergence", direction="min"),
                     max_epochs=3,
                     factor=3,
                     directory='results',
                     project_name='cwt_v1')


Reloading Tuner from results/cwt_v1/tuner0.json


In [ ]:
# tuner = kt.RandomSearch(
#     hypermodel=model_builder,
#     # objective="val_accuracy",
#     objective=kt.Objective("val_mean_absolute_error", direction="min"),
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="my_dir",
#     project_name="helloworld",
# )


In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}


In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_kullback_leibler_divergence', patience=5)


In [11]:
tuner.search(training_generator, epochs=3, validation_data=validation_generator, callbacks=[stop_early])

Trial 26 Complete [00h 00m 56s]
val_kullback_leibler_divergence: 1.3904061317443848

Best val_kullback_leibler_divergence So Far: 1.366502285003662
Total elapsed time: 19h 23m 12s


In [12]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 400, 32)       1472      
                                                                 
 conv2d_1 (Conv2D)           (None, 49, 400, 32)       9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 133, 32)       0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 16, 133, 32)       128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 16, 133, 32)       0         
                                                                 
 flatten (Flatten)           (None, 68096)             0

In [13]:
# tuner.results_summary()

Results summary
Results in results/cwt_v1
Showing 10 best trials
Objective(name="val_kullback_leibler_divergence", direction="min")

Trial 0012 summary
Hyperparameters:
num_layers: 2
units_0: 64
units_1: 64
units_2: 32
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0003
Score: 1.366502285003662

Trial 0020 summary
Hyperparameters:
num_layers: 2
units_0: 32
units_1: 64
units_2: 64
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0018
Score: 1.3739118576049805

Trial 0024 summary
Hyperparameters:
num_layers: 2
units_0: 64
units_1: 32
units_2: 32
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 1.3743195533752441

Trial 0018 summary
Hyperparameters:
num_layers: 2
units_0: 32
units_1: 64
units_2: 64
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 1.3895821571350098

Trial 0025 summary
Hyperparameters:
num_layers: 3
units_0: 64
units_1: 64
units_2: 64
tu

## Retrain the model

In [16]:
checkpoint_filepath = f'{output_dir}checkpoint-12-1.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # monitor='val_kl_divergence',
    monitor='val_kullback_leibler_divergence', 
    mode='min',
    save_best_only=True)

best_hps=tuner.get_best_hyperparameters()[0]

model = tuner.hypermodel.build(best_hps)

history = model.fit(training_generator, epochs=30,
          validation_data=validation_generator,
          callbacks=[model_checkpoint_callback])


Epoch 1/30
16/16 [==============================] - 5s 306ms/step - loss: 5.6126 - kullback_leibler_divergence: 5.6126 - val_loss: 1.8444 - val_kullback_leibler_divergence: 1.8444
Epoch 2/30
16/16 [==============================] - 5s 303ms/step - loss: 1.8072 - kullback_leibler_divergence: 1.8072 - val_loss: 1.3983 - val_kullback_leibler_divergence: 1.3983
Epoch 3/30
16/16 [==============================] - 5s 292ms/step - loss: 0.5965 - kullback_leibler_divergence: 0.5965 - val_loss: 1.3819 - val_kullback_leibler_divergence: 1.3819
Epoch 4/30
16/16 [==============================] - 5s 299ms/step - loss: 0.3470 - kullback_leibler_divergence: 0.3470 - val_loss: 1.3814 - val_kullback_leibler_divergence: 1.3814
Epoch 5/30
16/16 [==============================] - 5s 288ms/step - loss: 0.3340 - kullback_leibler_divergence: 0.3340 - val_loss: 1.3704 - val_kullback_leibler_divergence: 1.3704
Epoch 6/30
16/16 [==============================] - 5s 299ms/step - loss: 0.2715 - kullback_leibler_

## Scoring without submission

Using a local test set.

In [21]:
params = {
    'batch_size': 32,
    'n_classes': 6,
    }

test_generator = TestDataGenerator(path_test_items, path_test, **params)

y_pred = model.predict(test_generator)

4/4 [==============================] - 0s 35ms/step


In [22]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

test_items = np.load(path_test_items)
df_test_items = pd.DataFrame(test_items)
df_test_items[0] = df_test_items[0].astype(int)

sub = pd.DataFrame({'eeg_id':df_test_items[0]})
sub[TARGETS] = np.round(y_pred,6)
sub.to_csv('submission.csv',index=False)

df_test_scoring = df_test_items[[0,4,5,6,7,8,9]]
df_test_scoring.columns = sub.columns

score(df_test_scoring, sub, 'eeg_id')

/home/edg/hms/keras/../lib/KLmetric.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution[col] = solution[col].astype(float)
/home/edg/hms/keras/../lib/KLmetric.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution[col] = solution[col].astype(float)
/home/edg/hms/keras/../lib/KLmetric.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

1.6451832216960036